In [ ]:
pip install pymysql

In [ ]:
pip install spotipy

In [7]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pymysql
from datetime import datetime

In [6]:
# Spotify API 인증 설정
client_id = "9bd86ce2eea34cccb8961d47792e9796"  # Spotify Developer Dashboard에서 발급받은 Client ID
client_secret = "71c37f5b363a47ca8ce1248ed4f3dfaa"  # Spotify Developer Dashboard에서 발급받은 Client Secret

In [8]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [1]:
def get_track_id_from_url(url):
    """
    Spotify URL에서 트랙 ID 추출
    :param url: Spotify 트랙 URL
    :return: 트랙 ID
    """
    return url.split('/')[-1].split('?')[0]

In [3]:
def get_tracks_by_keyword(keyword, limit=5):
    """
    Spotify API를 통해 특정 키워드로 트랙 검색
    :param keyword: 검색 키워드 (e.g., "relaxing music")
    :param limit: 가져올 곡의 수
    :return: 곡 정보 리스트
    """
    results = sp.search(q=keyword, limit=limit, type="track") # 키워드 검색 해 트랙 가져오기
    tracks = [] # 트랙별 정보 딕셔너리가 담긴 리스트(name, )
    for item in results['tracks']['items']:
        # 트랙별 정보
        track_info = {
            "bgm_id": get_track_id_from_url(item['external_urls']['spotify']),  # URL에서 BGM ID 추출
            "bgm_name": item['name'],  # 곡 이름
            "bgm_url": item['external_urls']['spotify'],  # Spotify URL
            "artist": ", ".join([artist['name'] for artist in item['artists']]),  # 아티스트 이름들 (여러 명일 경우 ','로 연결)
            "album_cover": item['album']['images'][0]['url'] if item['album']['images'] else None,  # 앨범 커버 이미지 URL
        }
        tracks.append(track_info)  # 리스트에 추가
    return tracks



In [9]:
# 사용 예제
if __name__ == "__main__":
    keyword = "surprise"  # 검색 키워드
    tracks = get_tracks_by_keyword(keyword, limit=10)  # 상위 5곡 검색

    print("검색된 곡 정보:")
    for track in tracks:
        print(f"bgm 고유 아이디: {track['bgm_id']}곡 이름: {track['bgm_name']}, 아티스트: {track['artist']}, URL: {track['bgm_url']}, 앨범 커버: {track['album_cover']}")

검색된 곡 정보:
bgm 고유 아이디: 5CievkVz0QepORWuOsFHiu곡 이름: Surprise!, 아티스트: JANNABI, URL: https://open.spotify.com/track/5CievkVz0QepORWuOsFHiu, 앨범 커버: https://i.scdn.co/image/ab67616d0000b273847b562f6711f072dd2b11a6
bgm 고유 아이디: 10nyNJ6zNy2YVYLrcwLccB곡 이름: No Surprises, 아티스트: Radiohead, URL: https://open.spotify.com/track/10nyNJ6zNy2YVYLrcwLccB, 앨범 커버: https://i.scdn.co/image/ab67616d0000b273c8b444df094279e70d0ed856
bgm 고유 아이디: 40cWWy4RgszGtvmmiAaJLM곡 이름: 항복, 아티스트: MRCH, URL: https://open.spotify.com/track/40cWWy4RgszGtvmmiAaJLM, 앨범 커버: https://i.scdn.co/image/ab67616d0000b273e58b1f86bf1ddb97e3de1e0d
bgm 고유 아이디: 1ukI7i3fps8zaBlEf18v0l곡 이름: Surprise Yourself, 아티스트: Jack Garratt, URL: https://open.spotify.com/track/1ukI7i3fps8zaBlEf18v0l, 앨범 커버: https://i.scdn.co/image/ab67616d0000b27307143820b9a752eef7eb8643
bgm 고유 아이디: 3tDAE08rX2Rnsg7Hp943f1곡 이름: SURPRISE-DRIVE, 아티스트: Mitsuru Matsuoka EARNEST DRIVE, URL: https://open.spotify.com/track/3tDAE08rX2Rnsg7Hp943f1, 앨범 커버: https://i.scdn.co/image/ab676

In [7]:
def save_to_database(tracks):
    """
    곡 정보를 MySQL 데이터베이스에 저장
    :param tracks: 곡 정보 리스트
    :param bgm_emotion: 곡 감정 정보
    """
    connection = pymysql.connect(
        host="localhost",
        port=3306,
        user='root',
        password="1111",
        database="prome_15",
        charset="utf8mb4"
    ) # sql 커넥션
    try:
        with connection.cursor() as cursor:
            for track in tracks:
                sql = """
                INSERT INTO bgm_track (
                    bgm_id,bgm_name, bgm_url, artist, album_cover
                )
                VALUES (%s, %s, %s, %s, %s, %s)
                """
                cursor.execute(sql, (
                    track['bgm_id'],
                    track['bgm_name'],
                    track['bgm_url'],
                    track['artist'],
                    track['album_cover'],
                ))
        connection.commit()
    finally:
        connection.close()


In [8]:

if __name__ == "__main__":
    tracks = get_tracks_by_keyword("surpise", limit=5)
    save_to_database(tracks)
